# Scientific Data Analysis
Jasper Wink, 14616513 \
Dennis van der Werff, 14562189

# Imports

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import ast

from statsmodels.tsa.stattools import adfuller, grangercausalitytests
from statsmodels.tsa.api import VAR
import statsmodels.api as sm
import warnings

# There are irrelevant warning that make the output unreadable/less readable.
warnings.filterwarnings('ignore')

In [ ]:
# Read data
cn = pd.read_csv("data/cryptonews.csv")
bp = pd.read_csv("data/btc.csv")

# Prepare the data

In [ ]:
# Convert the date to the datetime format.
cn['date'] = pd.to_datetime(cn['date'], format='%Y-%m-%d %H:%M:%S')
bp['date'] = pd.to_datetime(bp['Date'], format='%Y-%m-%d')

# Discard unnecairy date data, eg. hours, minutes etc.
cn['weeks'] = cn['date'].dt.to_period('W')
bp['weeks'] = bp['date'].dt.to_period('W')

# Extract the polarity field out of the cn dataset.
cn['sentiment'] = cn['sentiment'].apply(ast.literal_eval)
cn['polarity'] = cn['sentiment'].apply(lambda x: x['polarity'])

# Group bitcoin data by the weeks and merge into new dataframe.
bp_weekly_open = bp.groupby('weeks')['Open'].mean().reset_index(name='open_mean')
bp_weekly_volume = bp.groupby('weeks')['Volume'].mean().reset_index(name='volume_mean')
bp_weekly_data = pd.merge(bp_weekly_open, bp_weekly_volume, on='weeks', how='inner')

# Group crpyto news data by the weeks and merge into new dataframe.
cn_weekly_count = cn.groupby('weeks').size().reset_index(name='count')
cn_weekly_sentiment = cn.groupby('weeks')['polarity'].mean().reset_index(name='polarity_mean')
cn_weekly_data = pd.merge(cn_weekly_count, cn_weekly_sentiment, on='weeks', how='inner')

# Merge all weekly data into 1 big dataframe.
weekly_data = pd.merge(bp_weekly_data, cn_weekly_data, on='weeks', how='inner')
print(weekly_data.head())


# Data Visualization

In [ ]:
# Normalize the data
count_list = list(weekly_data['count'])
normalized_count = [count / max(count_list) for count in count_list]

sentiment_list = list(weekly_data['polarity_mean'])
normalized_sentiment = [sentiment / max(sentiment_list) for sentiment in sentiment_list]

open_list = list(weekly_data['open_mean'])
normalized_price = [open / max(open_list) for open in open_list]

volume_list = list(weekly_data['volume_mean'])
normalized_volume = [volume / max(volume_list) for volume in volume_list]


x = np.linspace(0, len(normalized_count), len(normalized_count))
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Bitcoin Price vs Article Count
axes[0, 0].plot(x, normalized_price, label='Bitcoin price', color='red')
axes[0, 0].plot(x, normalized_count, label='Article count', color='orange')
axes[0, 0].set_title('Bitcoin price vs Article count')
axes[0, 0].set_xlabel('Weeks')
axes[0, 0].set_ylabel('Normalized values')
axes[0, 0].legend()

# Bitcoin Volume vs Article Count
axes[0, 1].plot(x, normalized_volume, label='Bitcoin volume', color='blue')
axes[0, 1].plot(x, normalized_count, label='Article count', color='orange')
axes[0, 1].set_title('Bitcoin volume vs Article count')
axes[0, 1].set_xlabel('Weeks')
axes[0, 1].set_ylabel('Normalized values')
axes[0, 1].legend()

# Bitcoin Price vs Sentiment
axes[1, 0].plot(x, normalized_price, label='Bitcoin price', color='red')
axes[1, 0].plot(x, normalized_sentiment, label='Sentiment', color='green')
axes[1, 0].set_title('Bitcoin Price vs Sentiment')
axes[1, 0].set_xlabel('Weeks')
axes[1, 0].set_ylabel('Normalized values')
axes[1, 0].legend()

# Bitcoin Volume vs Sentiment
axes[1, 1].plot(x, normalized_volume, label='Bitcoin volume', color='blue')
axes[1, 1].plot(x, normalized_sentiment, label='Sentiment', color='green')
axes[1, 1].set_title('Bitcoin volume vs Sentiment')
axes[1, 1].set_xlabel('Weeks')
axes[1, 1].set_ylabel('Normalized values')
axes[1, 1].legend()

plt.show()


# Stationary test

Using the Augmented Dickey-Fuller (ADF) test to check if the current data is satationary. \
If the p-value scores lower than 0.05 we reject the $H_0$ and determine that the data is stationary.

In [ ]:
# Test if the adfuller test scores lower than p-value = 0.05.
def test_stationary(data, name):
    ad_fuller_result = adfuller(data.dropna())
    if ad_fuller_result[1] > 0.05:
        print(f'{name} is not stationary | p-value: {ad_fuller_result[1]}')
    else:
        print(f'{name} is stationary | p-value: {ad_fuller_result[1]}')


# Prepair the data, by differentiating if needed. (Making it stationary)
weekly_data['open_diff'] = weekly_data['open_mean'].diff()
test_stationary(weekly_data['open_diff'], 'Bitcoin price')

weekly_data['volume_diff'] = weekly_data['volume_mean'].diff()
test_stationary(weekly_data['volume_diff'], 'Bitcoin volume')

test_stationary(weekly_data['count'], 'Article count')

test_stationary(weekly_data['polarity_mean'], 'News polarity')

# Stationary Data visualisation

In [ ]:
# Normalize the data
count_list = list(weekly_data['count'][1:])
normalized_count = [count / max(count_list) for count in count_list]

sentiment_list = list(weekly_data['polarity_mean'][1:])
normalized_sentiment = [sentiment / max(sentiment_list) for sentiment in sentiment_list]

open_list = list(weekly_data['open_diff'].dropna())
normalized_price = [open / max(open_list) for open in open_list]

volume_list = list(weekly_data['volume_diff'].dropna())
normalized_volume = [volume / max(volume_list) for volume in volume_list]


x = np.linspace(0, len(normalized_count), len(normalized_count))
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Bitcoin Price vs Article Count
axes[0, 0].plot(x, normalized_price, label='Bitcoin price', color='red')
axes[0, 0].plot(x, normalized_count, label='Article count', color='orange')
axes[0, 0].set_title('Bitcoin price vs Article count')
axes[0, 0].set_xlabel('Weeks')
axes[0, 0].set_ylabel('Normalized values')
axes[0, 0].legend()

# Bitcoin Volume vs Article Count
axes[0, 1].plot(x, normalized_volume, label='Bitcoin volume', color='blue')
axes[0, 1].plot(x, normalized_count, label='Article count', color='orange')
axes[0, 1].set_title('Bitcoin volume vs Article count')
axes[0, 1].set_xlabel('Weeks')
axes[0, 1].set_ylabel('Normalized values')
axes[0, 1].legend()

# Bitcoin Price vs Sentiment
axes[1, 0].plot(x, normalized_price, label='Bitcoin price', color='red')
axes[1, 0].plot(x, normalized_sentiment, label='Sentiment', color='green')
axes[1, 0].set_title('Bitcoin Price vs Sentiment')
axes[1, 0].set_xlabel('Weeks')
axes[1, 0].set_ylabel('Normalized values')
axes[1, 0].legend()

# Bitcoin Volume vs Sentiment
axes[1, 1].plot(x, normalized_volume, label='Bitcoin volume', color='blue')
axes[1, 1].plot(x, normalized_sentiment, label='Sentiment', color='green')
axes[1, 1].set_title('Bitcoin volume vs Sentiment')
axes[1, 1].set_xlabel('Weeks')
axes[1, 1].set_ylabel('Normalized values')
axes[1, 1].legend()

plt.show()


# Spearman Correlation
To be able to detect whether there is correlation between the bitcoin and the news we will be using spearman correlation. \
The correlation can score between -1 and 1 where 1 means a perfect correlation and -1 means a perfect inverse correlation. \
A score of 0 means there is no correlation.

In [ ]:
# https://en.wikipedia.org/wiki/Spearman's_rank_correlation_coefficient
def spearman_correlation(rank1, rank2):
    n = len(rank1)

    sum_d_squared = sum([(rank1[i] - rank2[i])**2 for i in range(n)])
    correlation = 1 - ((6 * sum_d_squared) / (n * (n**2 - 1)))
    return correlation


# Calculate ranks
bp_open_rank = list(weekly_data['open_diff'].dropna().rank())
bp_volume_rank = list(weekly_data['volume_diff'].dropna().rank())
cn_count_rank = list(weekly_data['count'][1:].rank())
cn_sentiment_rank = list(weekly_data['polarity_mean'][1:].rank())

pairs = [[bp_open_rank, cn_count_rank], [bp_volume_rank, cn_count_rank],
         [bp_open_rank, cn_sentiment_rank], [bp_volume_rank, cn_sentiment_rank]]

names = ['open price vs article count', 'volume vs article count',
         'open price vs sentiment', 'volume vs sentiment']

# Calculate the spearman correlation and print it.
for i, pair in enumerate(pairs):
    correlation = spearman_correlation(pair[0], pair[1])
    print(f'{names[i]}')
    print(f'Spearman Correlation: {correlation:.3f}\n')


# Vector AutoRegression

In [ ]:
def getVar(data1, data2, n, weekly_data):
    data = weekly_data[[data1, data2]].dropna()
    model = VAR(data)
    lags = {data1: [], data2: []}

    # Check what lags are significant for 0 to n max lags.
    for i in range(n):
        coef1 = []
        coef2 = []
        model_fit = model.fit(maxlags=i)

        coefs = model_fit.coefs
        pvalues = model_fit.pvalues_endog_lagged

        # Check if a lag is significant (p-value < 0.05).
        for j in range(i):
            if pvalues[j*2][0] < 0.05:
                coef1.append((j+1, coefs[j][0][0]))
            if pvalues[j*2+1][0] < 0.05:
                coef2.append((j+1, coefs[j][0][1]))

        # Store the significant lags and coeficients.
        lags[data1].append(coef1)
        lags[data2].append(coef2)

    return lags

In [ ]:
# Example to show output
data = weekly_data[['count', 'open_diff']].dropna()
model = VAR(data)
model_fit = model.fit(maxlags=5)
model_fit.summary()

$\textbf{The resulting equation for the example is as follows:}$ \
$C_t = 0.675 \cdot C_{t-1} + 0.004 \cdot O_{t-1} - 0.005 \cdot O_{t-2}$

# Akaike Information Criterion

In [ ]:
def getBestLag(data1, data2, n, weekly_data):
    min = (0, np.inf)
    lags = getVar(data1, data2, n, weekly_data)

    for i in range(n):
        x_columns_str = ''

        # Create all lag columns and multiply them by their coeficient.
        for key in lags.keys():

            for lag, coeff in lags[key][i]:
                weekly_data[f'{key}_t-{lag}'] = coeff * weekly_data[f'{key}'].shift(lag)
                x_columns_str += f'{key}_t-{lag} '

        # Create a list of selected column names so they can be selected.
        x_columns_list = x_columns_str.split(' ')[:-1]

        x = weekly_data[x_columns_list].dropna()
        y = weekly_data[data2][-len(x):]

        # Calculate the AIC using the Ordinairy Least Squares model.
        x = sm.add_constant(x)
        model = sm.OLS(y, x).fit()
        # Store the best (lowest scoring) model.
        min = min if min[1] <= model.aic else (i, model.aic)

    return min[0]


# Granger Causality Tests
To determine if one variable causes a change in another variable we are using Granger causality. \
If the p-value of a lag is lower than 0.05 we can determine that this lag influences the variable Thus Granger causes the other varible.

In [ ]:
# Use the grangercausalitytests to check if data 1 Granger Causes data2.
def granger_causality_test(data1, data2, max_VAR_lags):
    data = weekly_data[[data1, data2]].dropna()
    best_lag = getBestLag(data1, data2, max_VAR_lags, weekly_data)
    granger_result = grangercausalitytests(data, maxlag=best_lag, verbose=False)
    return granger_result


# Print if data 1 Granger Causes data 2.
def check_granger_cause(result, data1, data2):
    for _, result in result.items():
        p_value = result[0]['ssr_ftest'][1]
        if p_value < 0.05:
            print(f'+ {data1} Gragner Causes {data2}')
            return

    print(f'- {data1} does not Gragner Cause {data2}')

In [ ]:
# Example of grangercausalitytest show output
data = weekly_data[['count', 'open_diff']].dropna()
_ = grangercausalitytests(data, [5])

# Granger Causality Parsed

In [ ]:
combinations = [
    ['count', 'open_diff'],
    ['count', 'volume_diff'],
    ['polarity_mean', 'open_diff'],
    ['polarity_mean', 'volume_diff']
    ]
max_VAR_lags = 20

# Make a list with all combinations and their inverse combinations.
inverse_combinations = [[value[1], value[0]] for value in combinations]
all_combinations = combinations + inverse_combinations

# Find the best lags and test if data1 Granger Causes data 2.
for data1, data2 in all_combinations:
    granger_result = granger_causality_test(data1, data2, max_VAR_lags)
    check_granger_cause(granger_result, data1, data2)
